In [1]:
#%load_ext autoreload
#%autoreload 2

import copy
import pickle

import matplotlib.pyplot as plt
import numpy as np
import tqdm

from src.benchmark import run_benchmark

### Manual run

In [2]:
# start a single meshcat server instance to use for remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])
print(web_url)

http://127.0.0.1:7000/static/


In [3]:
%%time

run_benchmark(
    zmq_url=zmq_url,
    P_WORLD_TARGET=np.array([-1, 1, -3]),
    MAX_APPROACH_ANGLE=-45 / 180.0 * np.pi,
    OBJECT_TO_TOSS="ball",
    GRIPPER_TO_OBJECT_COM_DIST=0.11,
    LAUNCH_ANGLE_THRESH=-3 / 180.0 * np.pi, # 3 seems to work well?
    verbose=False,
)

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connected to meshcat-server.
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connected to meshcat-server.
CPU times: user 17.6 s, sys: 1.51 s, total: 19.1 s
Wall time: 20.9 s


{'land_pos_error': 0.12804023168115441,
 'land_angle_error': 0.01541542977444621,
 'aim_angle_error': 1.7844357312668535e-05,
 'time_to_arrival': 0.9300237339644777,
 'land_time': 11.153,
 'land_x': -1.0905362795131577,
 'land_y': 1.090497360369573,
 'land_z': -2.997223169609295,
 'sim_land_angle': -1.428542694047865,
 'plan_land_angle': -1.4439581238223111,
 'plan_launch_angle': 0.6727153661238747,
 'release_frac': 0.2557226330825729,
 'throw_motion_time': 0.8289776432601338}

# Data time

In [4]:
def go(kwargs):
    ret = kwargs.copy()
    
    try:
        rb = run_benchmark(
            P_WORLD_TARGET=np.array([
                kwargs["X"],
                kwargs["Y"],
                kwargs["Z"]
            ]),
            **kwargs
        )
        ret.update(rb)
        ret.update(status="OK")
    except:
        ret.update(status="CRASH")
    
    return ret

In [5]:
MAX_APPROACH_ANGLE = -45 / 180.0 * np.pi
OBJECT_TO_TOSS = "ball"
GRIPPER_TO_OBJECT_COM_DIST = 0.11

kwargs_to_run = []
for launch_angle_thresh in np.array([-3, 0, 3, 6]) / 180.0 * np.pi:
    for z in [-3, -2, -1, 0, 1, 2, 3]:
        for x in [-1, -2, -3, -4, -5]:
            for y in [1, 2, 3, 4, 5]:
                kwargs_to_run.append(dict(
                    X=x,
                    Y=y,
                    Z=z,
                    LAUNCH_ANGLE_THRESH=launch_angle_thresh,
                    MAX_APPROACH_ANGLE=MAX_APPROACH_ANGLE,
                    OBJECT_TO_TOSS=OBJECT_TO_TOSS,
                    GRIPPER_TO_OBJECT_COM_DIST=GRIPPER_TO_OBJECT_COM_DIST,
                ))

print(len(kwargs_to_run))
print(f"Projected runtime: {20 * len(kwargs_to_run) / 60 / 60} hours")

700
Projected runtime: 3.888888888888889 hours


In [6]:
%%time

results = []
for kwargs in tqdm.tqdm(kwargs_to_run):
    cur_result = go(kwargs)
    results.append(cur_result)
    
    with open('benchmark_checkpoint.pickle', 'wb') as f:
        pickle.dump(results, f)

100%|██████████| 3/3 [01:00<00:00, 20.21s/it]

CPU times: user 46.3 s, sys: 230 ms, total: 46.5 s
Wall time: 1min


In [7]:
with open('benchmark_checkpoint.pickle', 'rb') as f:
    test = pickle.load(f)
    print(len(test))
    print(test[0])

3
{'X': -1, 'Y': 1, 'Z': -3, 'LAUNCH_ANGLE_THRESH': -0.05235987755982988, 'MAX_APPROACH_ANGLE': -0.7853981633974483, 'OBJECT_TO_TOSS': 'ball', 'GRIPPER_TO_OBJECT_COM_DIST': 0.11, 'land_pos_error': 0.12804023168114345, 'land_angle_error': 0.01541542977446464, 'aim_angle_error': 1.784435731533307e-05, 'time_to_arrival': 0.9300237339644777, 'land_time': 11.153, 'land_x': -1.0905362795131521, 'land_y': 1.090497360369561, 'land_z': -2.99722316960923, 'sim_land_angle': -1.4285426940478465, 'plan_land_angle': -1.4439581238223111, 'plan_launch_angle': 0.6727153661238747, 'release_frac': 0.2557226330825729, 'throw_motion_time': 0.8289776432601338, 'status': 'OK'}
